## PFT

### Training the model on FIA dataset

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Libraries for modeling
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report 
from xgboost import XGBClassifier
import time
from shapely.geometry import Point
import geopandas as gpd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.compose import ColumnTransformer

In [ ]:
# Reading the data file - FIATreeSpeciesCode_pft.csv
df_fia_species = pd.read_csv("./data/FIATreeSpeciesCode_pft.csv", sep=";")
df_fia_species_imp = df_fia_species[['SPCD', 'COMMON_NAME', 'PFT']].copy()

# Reading the data file - CA_TREE.csv
df_fia_tree = pd.read_csv("./data/CA_TREE.csv", low_memory=False)
df_fia_tree_imp = df_fia_tree[['STATECD', 'PLOT', 'PLT_CN', 'UNITCD', 'COUNTYCD', 'TREE', 'SPCD', 'DIA', 'HT', 'CR']].copy()

# Reading the data file - CA_PLOT.csv
df_fia_plot = pd.read_csv("./data/CA_PLOT.csv", low_memory=False)
df_fia_plot_imp = df_fia_plot[['PLOT_STATUS_CD', 'LAT', 'LON', 'ELEV', 'ECOSUBCD', 'CN']].copy()

# Analyzing the shape
print(df_fia_species_imp.shape)
print(df_fia_tree_imp.shape)
print(df_fia_plot_imp.shape)

In [ ]:
# Merging the FIA Species and FIA tree based on SPCD
df_fia = df_fia_tree_imp.merge(df_fia_species_imp, on="SPCD", how="left")
# Merging the resulting FIA dataset and FIA plot based on PLOT_CN and CN
df_fia = df_fia.merge(df_fia_plot_imp, left_on="PLT_CN", right_on="CN", how="left")
print(df_fia.shape)
df_fia.head()

In [ ]:
# Convert inches to cm
df_fia['DIA'] = df_fia['DIA'] * 2.54 
# Calculating the basal area
df_fia['BasalA'] = (np.pi * df_fia['DIA']**2) / (4 * 144)
df_fia.head()

In [ ]:
# Analyzing the distribution of PFT
df_fia['PFT'].value_counts()

As we can observe the data is particularly skewed towards Evergreen conifer. However, we wanted to develop a general model particularly focusing on the sites

    "Shaver Lake (SHA)": "M261Ep",

    "Sedgwick Reserve (SDR)": "261Ba",

    "Calaveras Big Trees State Park": "M261Em",

    "Pacific Union College (PUC)": "263Am",

    "Independence Lake (IND)": "M261Ej",

    "Winton-Schaads VMP (WIN)" :"M261Em",

So we need to filter these sites based on ECOSUBCD, but first let's select the important features

In [ ]:
# Selecting the important features
df_fia_important_features = df_fia[['DIA', 'HT', 'BasalA', 'LAT', 'LON', 'COMMON_NAME', 'SPCD', 'ECOSUBCD', 'PFT']].copy()
df_fia_important_features.head()

In [ ]:
# Dropping rows with missing values
print("Shape before:", df_fia_important_features.shape)
df_fia_important_features = df_fia_important_features.dropna(axis=0).copy()
print("Shape before:", df_fia_important_features.shape)
df_fia_important_features.head()

In [ ]:
# Shortlisting the data based on the sites to focus on

# Define the ECOSUBCDs to keep
ecosubcd_keep = ['M261Ep', '261Ba', 'M261Em', '263Am', 'M261Ej']

# Filter the DataFrame
df_fia_filtered = df_fia_important_features[df_fia_important_features['ECOSUBCD'].isin(ecosubcd_keep)]
# Shuffle the filtered DataFrame
df_fia_filtered_shuffled = df_fia_filtered.sample(frac=1, random_state=42).reset_index(drop=True)

# Keeping the original ECOSUBCD column for later use
df_fia_ecosubcd = df_fia_filtered_shuffled['ECOSUBCD']

# One hot encoding the categorical variable ECOSUBCD
df_fia_encoded = pd.get_dummies(df_fia_filtered_shuffled, columns=['ECOSUBCD'])
df_fia_encoded = df_fia_encoded.drop(['ECOSUBCD_M261Ep'], axis=1)
df_fia_encoded['ECOSUBCD'] = df_fia_ecosubcd
df_fia_encoded.head()

In [ ]:
# Making the corrections to the PFT column based on the email
replace_dict = {'Deciduous': 'Deciduous broadleaf'} 
df_fia_encoded['PFT'] = df_fia_encoded['PFT'].replace(replace_dict)
df_encoded = df_fia_encoded[df_fia_encoded['PFT'] != 'Broadleaf']
df_encoded['PFT'].value_counts()

In [ ]:
# Analyzing the distribution of PFT across different ECOSUBCDs
df_encoded.groupby('ECOSUBCD')['PFT'].value_counts(normalize=True) * 100

A lot of sites are skewed towards a particular PFT type, so location plays an important role in this distribution. When we consider this to our intuition, it makes sense as certain types of trees are located in the certain areas.

We will now transform the data for training and train the model

In [ ]:
df_clean = df_encoded.copy()

# Defining the independent and dependent variables
independent_variables = ['DIA', 'HT', 'BasalA', 'LAT', 'LON',
                         'ECOSUBCD_261Ba', 'ECOSUBCD_263Am',
                         'ECOSUBCD_M261Ej', 'ECOSUBCD_M261Em']
dependent_variable = "PFT"
include_variables = independent_variables + [dependent_variable]

# Separate features (X) and target (y)
X = df_clean[independent_variables]
y = df_clean[dependent_variable]

# Printing the class distribution
print("Class distribution in the dataset:")
print(y.value_counts(normalize=True))

# Encode target variable
le = LabelEncoder()
y_encoded = le.fit_transform(y)
num_classes = len(le.classes_)
print(f"Encoded classes: {list(le.classes_)}")
print(f"Number of classes: {num_classes}")

# Split into training and test sets
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(
    X, y_encoded, test_size=0.25, random_state=42, stratify=y_encoded
)

print(f"Original Training set size: {len(X_train)}")
print(f"Original Test set size: {len(X_test)}")

# Feature Scaling
numerical_cols = ['DIA','HT', 'BasalA', 'LAT', 'LON']
categorical_cols = [col for col in independent_variables if col not in numerical_cols]

scaler = StandardScaler()

# Scale numerical features
X_train_scaled_num = scaler.fit_transform(X_train[numerical_cols])
X_test_scaled_num = scaler.transform(X_test[numerical_cols])

X_train_processed = np.hstack((X_train_scaled_num, X_train[categorical_cols].values))
X_test_processed = np.hstack((X_test_scaled_num, X_test[categorical_cols].values))

# Stratified K-Folds cross-validator
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train_processed, y_train_encoded, test_size=0.30, random_state=42, stratify=y_train_encoded
)
start_time = time.time()

# --- Best Base Models ---

rf = RandomForestClassifier(
    n_estimators=317,
    max_depth=23,
    min_samples_split=6,
    min_samples_leaf=1,
    max_features=None,
    bootstrap=True,
    criterion='log_loss',
    class_weight='balanced',
    random_state=42
)

et = ExtraTreesClassifier(
    n_estimators=317,
    max_depth=23,
    min_samples_split=6,
    min_samples_leaf=1,
    max_features=None,
    bootstrap=True,
    criterion='log_loss',
    random_state=42
)

dt = DecisionTreeClassifier(
    max_depth=14,
    min_samples_split=13,
    min_samples_leaf=3,
    max_features=None,
    criterion='gini',
    splitter='best',
    random_state=42
)

bagging = BaggingClassifier(
    n_estimators=71,
    max_samples=0.616,
    max_features=0.987,
    bootstrap=False,
    bootstrap_features=False,
    random_state=42
)

# --- Meta-Learner ---
meta_learner = XGBClassifier(
    n_estimators=180,
    max_depth=13,
    learning_rate=0.0116,
    subsample=0.803,
    colsample_bytree=0.886,
    gamma=0.994,
    reg_alpha=0.711,
    reg_lambda=0.790,
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42
)

# --- Stacking Ensemble ---
final_model = StackingClassifier(
    estimators=[
        ('rf', rf),
        ('et', et),
        ('dt', dt),
        ('bagging', bagging)
    ],
    final_estimator=meta_learner,
    cv=skf,
    n_jobs=-1,
    passthrough=True
)

final_model.fit(
    X_train_final,
    y_train_final,
)

print(f"Final model training finished. Time: {time.time() - start_time:.2f} seconds")
print("-" * 30)

# --- Predict and Evaluate on the Test Set ---
print("Evaluating model on the test set...")

# Predict using the final model
predicted_pft_test_encoded = final_model.predict(X_test_processed)

# Decode predictions for readability
predicted_labels_decoded = le.inverse_transform(predicted_pft_test_encoded)

# Get original test labels for comparison
original_test_labels = le.inverse_transform(y_test_encoded)

# Classification report
report = classification_report(
    original_test_labels,
    predicted_labels_decoded,
    zero_division=0,
    target_names=le.classes_
)
print("Classification Report on Test Set:")
print(report)

A lot of sites are skewed towards a particular PFT, so location might play an important role in this distribution. When we consider this to our intuition, it makes sense as certain types of trees are located in the certain areas.

### Working with TLS Data for prediction

In [ ]:
# Loading the TLS data
tls_treelist_df = pd.read_csv("./data/TLS_treelist.csv", index_col=0)
df_tls_plot_identification = pd.read_csv("./data/blk_plot_identification.csv")
df_tls_data = tls_treelist_df.merge(df_tls_plot_identification,on="plot_blk", how="left")
df_tls_data.head()

In [ ]:
# Checking the distribution of the site names
df_tls_data['site_name_label'].value_counts()

In [ ]:
print("Shape before:",df_tls_data.shape)
# Dropping the null values
df_tls_data = df_tls_data.dropna(axis=0).copy()
print("Shape after:",df_tls_data.shape)

In [ ]:
# Converting X, Y coordinates to Latitude and Longitude
gdfs = []
for crs in df_tls_data["plot_coord_srs"].unique():
    df_crs = df_tls_data[df_tls_data["plot_coord_srs"] == crs]
    geometry = [Point(xy) for xy in zip(df_crs["X"], df_crs["Y"])]
    gdf = gpd.GeoDataFrame(df_crs, geometry=geometry, crs=f"EPSG:{crs}")
    gdf = gdf.to_crs(epsg=4326)
    gdfs.append(gdf)

# Recombine the GeoDataFrames
gdf = pd.concat(gdfs)
gdf["longitude"] = gdf.geometry.x
gdf["latitude"] = gdf.geometry.y

df_tls_data['LAT'] = gdf['latitude']
df_tls_data['LON'] = gdf['longitude']

In [ ]:
# Now we will map the site names to the corresponding ECOSUBCDs
#  Define mapping
site_to_ecosubcd = {
    "Shaver Lake (SHA)": "M261Ep",
    "Sedgwick Reserve (SDR)": "261Ba",
    "Calaveras Big Trees State Park": "M261Em",
    "Pacific Union College (PUC)": "263Am",
    "Independence Lake (IND)": "M261Ej",
    "Winton-Schaads VMP (WIN)" :"M261Em",
}

# Create Ecosubcd column based on site names
df_tls_data['ECOSUBCD'] = df_tls_data['site_name_label'].map(site_to_ecosubcd)

In [ ]:
# Selecting important features from the TLS data
TLS_X = df_tls_data[['DBH', 'H', 'BasalA', 'LAT', 'LON', 'ECOSUBCD']].copy()
# Renaming columns for consistency
TLS_X.columns = ['DIA', 'HT', 'BasalA', 'LAT', 'LON', 'ECOSUBCD']

In [ ]:
# One hot encoding the categorical variable ECOSUBCD
TLS_X_ECOSUBCD = TLS_X['ECOSUBCD'].copy()
TLS_X = pd.get_dummies(TLS_X, columns=['ECOSUBCD'])
# Dropping one of the dummy variables to avoid the dummy variable trap
TLS_X = TLS_X.drop(['ECOSUBCD_M261Ep'], axis=1)
TLS_X = TLS_X[['DIA', 'HT', 'BasalA', 'LAT', 'LON',
                         'ECOSUBCD_261Ba', 'ECOSUBCD_263Am',
                         'ECOSUBCD_M261Ej', 'ECOSUBCD_M261Em']]
TLS_X['ECOSUBCD'] = TLS_X_ECOSUBCD

# Shuffle the filtered DataFrame
TLS_X_shuffled = TLS_X.sample(frac=1, random_state=42).reset_index(drop=True)
TLS_X_shuffled = TLS_X.copy()

# dropping the null values
TLS_X_shuffled = TLS_X_shuffled.dropna(axis=0).copy()
TLS_X_shuffled.head()

In [ ]:
# Defining the independent variables

independent_variables = ['DIA','HT', 'BasalA', 'LAT', 'LON',
                         'ECOSUBCD_261Ba', 'ECOSUBCD_263Am',
                         'ECOSUBCD_M261Ej', 'ECOSUBCD_M261Em']

numerical_cols = ['DIA', 'HT', 'BasalA', 'LAT', 'LON']
categorical_cols = [col for col in independent_variables if col not in numerical_cols]

In [ ]:
# Scale numerical features
TLS_X_scaled = scaler.fit_transform(TLS_X_shuffled[numerical_cols])
TLS_train_processed = np.hstack((TLS_X_scaled, TLS_X_shuffled[categorical_cols].values))

# Predicting the PFTs using the final model
predicted_pft_tls = final_model.predict(TLS_train_processed)
# Decode predictions for readability
predicted_labels_TLS_decoded = le.inverse_transform(predicted_pft_tls)
print("Length of predicted PFTs:",len(predicted_labels_TLS_decoded))

# Adding the predicted PFTs to the TLS data
TLS_X['predicted_PFT'] = predicted_labels_TLS_decoded
print(TLS_X['predicted_PFT'].value_counts())
print("-" * 30)

# Calculate value counts as percentages
percentages = TLS_X['predicted_PFT'].value_counts(normalize=True) * 100

print("-" * 30)
print("Percentage distribution of predicted PFTs:")
# Round and display
percentages = percentages.round(2)
print(percentages)

In [ ]:
# Calculate value counts as percentages
percentages = TLS_X.groupby('ECOSUBCD')['predicted_PFT'].value_counts(normalize=True) * 100

# Round and display
percentages = percentages.round(2)
print(percentages)

### Working with Field_data

Now we will load the Field data to compare our predictions

In [ ]:
df_field = pd.read_csv("./data/03_tree.csv")
df_field_plot = pd.read_csv("./data/01_plot_identification.csv")

print(df_field.shape)
print(df_field_plot.shape)

In [ ]:
# Identifying the scientific names in the field data that are not mapped with the FIA data
field_names = set(df_field['tree_sp_scientific_name'].value_counts().index)
fia_names = set(df_fia_species['SCI_NAME'].value_counts().index)
difference = list(field_names - fia_names)
difference

In [ ]:
# Creating a mapping for the differences to maintain uniformity
replace_dict = {'Salix spp.': 'Salix sp.', 'Quercus spp.': 'Quercus sp.', 'Cornus nuttallii': 'Cornus nuttalii'} 
df_fia_species['SCI_NAME'] = df_fia_species['SCI_NAME'].replace(replace_dict)

In [ ]:
# Merging the field data with the plot data and then with the FIA species data
df_field = df_field.merge(df_field_plot, on="inventory_id", how="left")
df_field = df_field.merge(df_fia_species, left_on="tree_sp_scientific_name", right_on="SCI_NAME", how="left")
print("Field data shape:",df_field.shape)
df_field.head()

In [ ]:
# Replacing Deciduous with Deciduous broadleaf as both are the same
replace_dict = {'Deciduous': 'Deciduous broadleaf'} 
df_field['PFT'] = df_field['PFT'].replace(replace_dict)

In [ ]:
# Calculate value counts as percentages
percentages = df_field['PFT'].value_counts(normalize=True) * 100

# Round and display
percentages = percentages.round(2)
print("Field data PFT distribution:")
print(percentages)
print("-" * 30)

print("Predicted TLS data distribution:")
percentages = TLS_X['predicted_PFT'].value_counts(normalize=True) * 100
percentages = percentages.round(2)
print(percentages)

Here, we can see that the overall distribution from our model is pretty close to the actual distribution of the site data, with Evergreen broadleaf distribution being almost the same

In [ ]:
# Define your mapping
site_to_ecosubcd = {
    "Shaver Lake (SHA)": "M261Ep",
    "Sedgwick Reserve (SDR)": "261Ba",
    "Calaveras Big Trees State Park": "M261Em",
    "Pacific Union College (PUC)": "263Am",
    "Independence Lake (IND)": "M261Ej",
    "Winton-Schaads VMP (WIN)" :"M261Em",
}

# Create new column
df_field['ECOSUBCD'] = df_field['site_name_label'].map(site_to_ecosubcd)
# Calculate value counts as percentages
percentages = df_field.groupby('ECOSUBCD')['PFT'].value_counts(normalize=True) * 100

# Round and display
percentages = percentages.round(2)
print(percentages)

In [ ]:
# Calculate value counts as percentages
percentages = TLS_X.groupby('ECOSUBCD')['predicted_PFT'].value_counts(normalize=True) * 100

# Round and display
percentages = percentages.round(2)
print(percentages)

Here we can see that even the site wise distributions are pretty close as well

## GENUS

In [ ]:
# Creating a copy of the above dataframe for the FIA genus
df_fia_genus = df_encoded.copy()
df_fia_genus.head()

In [ ]:
# Loading the reference species data to get info on GENUS and SPECIES
df_ref_species = pd.read_csv('./data/REF_SPECIES.csv')
df_ref_species = df_ref_species[['SPCD', 'GENUS', 'SPECIES', 'COMMON_NAME']].copy()
print(f"Shape: {df_ref_species.shape}")
df_ref_species.head()

In [ ]:
# Merging the FIA data with the reference species data
df_fia_genus = df_fia_genus.merge(df_ref_species, on="SPCD", how="left")
print(f"Shape: {df_fia_genus.shape}")
df_fia_genus.head()

In [ ]:
# List of GENUS to keep
genus_to_keep = ['Abies', 'Calocedrus', 'Pinus', 'Cornus', 'Populus',
                  'Quercus', 'Sequoiadendron', 'Acer', 'Pseudotsuga', 'Arbutus','Salix'
                  'Notholicarpos']

# Filter the DataFrame
df_fia_genus = df_fia_genus[df_fia_genus["GENUS"].isin(genus_to_keep)].copy()
df_fia_genus.shape

In [ ]:
# # Define a function to remove outliers based on IQR
def remove_outliers_by_genus(df, column, group_by):
    Q1 = df.groupby(group_by)[column].transform('quantile', 0.25)
    Q3 = df.groupby(group_by)[column].transform('quantile', 0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

In [ ]:
# Check the shape
print(f"Shape before removing outliers: {df_fia_genus.shape}")

# Remove outliers based on GENUS
columns = ['DIA', 'HT', 'BasalA', 'LAT', 'LON']

for column in columns:
    df_fia_genus = remove_outliers_by_genus(df_fia_genus, column, 'GENUS')

# Check the resulting shape
print(f"Shape after removing outliers: {df_fia_genus.shape}")

In [ ]:
# make box plots to see outliers 
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.boxplot(x='GENUS', y='DIA', data=df_fia_genus)
plt.xticks(rotation=90)
plt.title('Boxplot of DIA_cm by GENUS')
plt.show()
plt.figure(figsize=(10, 6))

sns.boxplot(x='GENUS', y='HT', data=df_fia_genus)
plt.xticks(rotation=90)
plt.title('Boxplot of HT by GENUS')
plt.show()
plt.figure(figsize=(10, 6))
sns.boxplot(x='GENUS', y='BasalA', data=df_fia_genus)
plt.xticks(rotation=90)
plt.title('Boxplot of BasalA by GENUS')
plt.show()
plt.figure(figsize=(10, 6))
sns.boxplot(x='GENUS', y='LAT', data=df_fia_genus)
plt.xticks(rotation=90)
plt.title('Boxplot of LAT by GENUS')
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='GENUS', y='LON', data=df_fia_genus)
plt.xticks(rotation=90)
plt.title('Boxplot of LON by GENUS')
plt.show()

In [ ]:
# Typecasting PFT as cateogorical column
df_fia_genus['PFT'] = df_fia_genus['PFT'].astype('category')

# selecting important features
include_vars = ['DIA', 'HT', 'LAT', 'LON', 'ECOSUBCD_261Ba', 'ECOSUBCD_263Am',
                'ECOSUBCD_M261Ej', 'ECOSUBCD_M261Em', 'PFT', 'GENUS']
df = df_fia_genus[include_vars].dropna().copy()

# Create the encoder
ohe = OneHotEncoder(sparse_output=False)

# Fit and transform the PFT column
pft_encoded = ohe.fit_transform(df[["PFT"]])

# Convert to DataFrame with appropriate column names
pft_encoded_df = pd.DataFrame(
    pft_encoded,
    columns=[f"PFT_{cat}" for cat in ohe.categories_[0]],
    index=df.index
)

# Drop the original PFT column and join the encoded columns
df = df.drop(columns=["PFT"])
df = pd.concat([df, pft_encoded_df], axis=1)

df.rename(columns={"DIA": "DIA"}, inplace=True)
X = df.drop(columns=["GENUS"])
y = df["GENUS"]

# Encode target variable
le = LabelEncoder()
y_encoded = le.fit_transform(y)
num_classes = len(le.classes_)


# Stratified train-test split
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(
    X, y_encoded, test_size=0.30, stratify=y_encoded , random_state=42,
)


# Identify numerical and categorical columns again
numerical_cols = ['DIA', 'HT', 'LAT', 'LON']
categorical_cols = [col for col in X.columns if col not in numerical_cols]

# Create column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_cols),
    ]
)

# Fit and transform training data
X_train_scaled = preprocessor.fit_transform(X_train)
X_test_scaled = preprocessor.transform(X_test)


# Concatenate scaled numerical and raw categorical columns
X_train_processed = np.hstack((X_train_scaled, X_train[categorical_cols].values))
X_test_processed = np.hstack((X_test_scaled, X_test[categorical_cols].values))

In [ ]:
# Checking the distribution
print(df_fia_genus["GENUS"].value_counts())
print("Len :",len(df_fia_genus["GENUS"].unique()))

In [ ]:
# Assume X, y_encoded are your full datasets with the same number of rows
assert len(X) == len(y_encoded), "X and y_encoded must have the same number of rows."

# Compute sample weights
sample_weights = compute_sample_weight(class_weight='balanced', y=y_encoded)

# Check all shapes
print(f"X: {X.shape}, y: {len(y_encoded)}, sample_weights: {len(sample_weights)}")

In [ ]:
# Using the earlier column transformer to preprocess the data
X_new = np.hstack((preprocessor.fit_transform(X), X[categorical_cols].values))

In [ ]:
# Tuning the Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import time

# Best hyperparameters found earlier
best_rf_model = RandomForestClassifier(
    class_weight={
        0: 0.4252973600232086,
        1: 28.745098039215687,
        2: 10.471428571428572,
        3: 0.4328314142308828,
        4: 18.325,
        5: 0.36314094624721327,
        6: 77.15789473684211,
        7: 3.34703196347032,
        8: 0.4827132038195588,
        9: 61.083333333333336
    },
    criterion='entropy',
    max_depth=100,
    min_samples_split=5,
    n_estimators=800,
    random_state=42
)

# Evaluation function
def train_and_evaluate(model, name):
    start_time = time.time()
    model.fit(X_train_processed, y_train_encoded)
    predictions = model.predict(X_test_processed)
    accuracy = accuracy_score(y_test_encoded, predictions)
    report = classification_report(y_test_encoded, predictions)
    end_time = time.time()
    training_time = end_time - start_time
    print(f"{name} - Accuracy: {accuracy:.4f}, Training Time: {training_time:.2f} seconds")
    print(report)
    print("-" * 50)

# Train and evaluate the best RF model
train_and_evaluate(best_rf_model, "Best Random Forest")

In [ ]:
# Tuning the Extra Trees Classifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report, accuracy_score
import time

# Best hyperparameters from tuning
best_et_model = ExtraTreesClassifier(
    n_estimators=800,
    min_samples_split=2,
    min_samples_leaf=2,
    max_features=None,
    max_depth=30,
    criterion='entropy',
    bootstrap=False,
    random_state=42
)

# Evaluation function (already defined)
def train_and_evaluate(model, name):
    start_time = time.time()
    model.fit(X_train_processed, y_train_encoded)
    predictions = model.predict(X_test_processed)
    accuracy = accuracy_score(y_test_encoded, predictions)
    report = classification_report(y_test_encoded, predictions)
    end_time = time.time()
    training_time = end_time - start_time
    print(f"{name} - Accuracy: {accuracy:.4f}, Training Time: {training_time:.2f} seconds")
    print(report)
    print("-" * 50)

# Train and evaluate the final Extra Trees model
train_and_evaluate(best_et_model, "Best Extra Trees")

In [ ]:
# Tuning the Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
import time

# Best hyperparameters from tuning
best_dt_model = DecisionTreeClassifier(
    splitter='best',
    min_samples_split=15,
    min_samples_leaf=4,
    max_features=None,
    max_depth=50,
    criterion='entropy',
    random_state=42
)

# Evaluation function (already defined)
def train_and_evaluate(model, name):
    start_time = time.time()
    model.fit(X_train_processed, y_train_encoded)
    predictions = model.predict(X_test_processed)
    accuracy = accuracy_score(y_test_encoded, predictions)
    report = classification_report(y_test_encoded, predictions)
    end_time = time.time()
    training_time = end_time - start_time
    print(f"{name} - Accuracy: {accuracy:.4f}, Training Time: {training_time:.2f} seconds")
    print(report)
    print("-" * 50)

# Train and evaluate the final Decision Tree model
train_and_evaluate(best_dt_model, "Best Decision Tree")

In [ ]:
# Tuning the meta learner - XGBoost
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier

# Define base models (already trained or initialized)
base_models = [
    ('rf', best_rf_model),
    ('et', best_et_model),
    ('dt', best_dt_model)
]

# Use best config from FLAML to define the meta-learner
best_xgb_meta = XGBClassifier(
    n_estimators=50,
    learning_rate=0.14,
    max_depth=3,
    min_child_weight=3,
    subsample=0.86,
    colsample_bytree=0.9,
    gamma=0.064,
    reg_alpha=0.068,
    reg_lambda=1.4,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Define the final stacking classifier
final_stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=best_xgb_meta,
    cv=5,
    n_jobs=-1
)

# Fit the stacking model
final_stack_model.fit(X_train_processed, y_train_encoded)

# Evaluate
train_and_evaluate(final_stack_model, "Stacking with Best-Tuned XGBoost")


In [ ]:
# Visualizing the final stacked model
final_stack_model

In [ ]:
# Fitting the data to final stacked model
final_stack_model.fit(X_new,y_encoded, sample_weight=sample_weights)

### Working with TLS data

In [ ]:
# We rename the predicted_PFT column to PFT for uniformity
TLS_X_genus = TLS_X.copy()
TLS_X_genus.columns = ['DIA', 'HT', 'BasalA', 'LAT', 'LON','ECOSUBCD_261Ba','ECOSUBCD_263Am', 'ECOSUBCD_M261Ej', 'ECOSUBCD_M261Em', 'ECOSUBCD', 'PFT']

In [ ]:
# One-hot encode the PFT column
PFT_encoded = pd.get_dummies(TLS_X_genus['PFT'], prefix='PFT')

# Concatenate the one-hot encoded columns with the original DataFrame
TLS_X_genus = pd.concat([TLS_X_genus, PFT_encoded], axis=1)

# Drop the PFT column
TLS_X_genus.drop(columns=['PFT'], inplace=True)

TLS_X_genus['PFT_Evergreen broadleaf'] = ~TLS_X_genus['PFT_Deciduous broadleaf'] & ~TLS_X_genus['PFT_Evergreen conifer']

# Display the updated DataFrame
TLS_X_genus.head()

In [ ]:
# concatenate the scaled numerical features with the categorical features
TLS_X_cat_n_num_cols = np.hstack((preprocessor.fit_transform(TLS_X_genus), TLS_X_genus[categorical_cols].values))

# Predict using the final stacked model
pred =  final_stack_model.predict(TLS_X_cat_n_num_cols)
predicted_genus = le.inverse_transform(pred)

# Adding the predicted GENUS to the TLS data
TLS_X_genus['predicted_genus'] = predicted_genus.copy()

# checking the distribution
TLS_X_genus['predicted_genus'].value_counts()

### Working with field data

In [ ]:
df_field[['GENUS', 'SPECIES']] = df_field['tree_sp_scientific_name'].str.split(' ', n=1, expand=True)
df_field.head()

In [ ]:
df_fia_genus['GENUS'].value_counts(normalize=True) * 100

In [ ]:
# find percentage of each predicted genus
predicted_genus_counts = TLS_X_genus['predicted_genus'].value_counts(normalize=True) * 100
# Display the percentage of each predicted genus
print("Percentage of each predicted genus:")
print(predicted_genus_counts)

In [ ]:
df_field['GENUS'].value_counts(normalize=True) * 100

Here, we have predicted Pinus as in training data distribution, Pinus is indeed a predominant Genus, while Abies's distribution is close enought at 19% (Predicted) to 23% (Field data) and Quercus at 20% (Predicted) to 21% (Field data). To predict Genus accurately, more tree specific features could help make model more accurate

## SPECIES

In [ ]:
# Creating a copy of the above dataframe for the FIA genus
TLS_species = TLS_X_genus.copy()
df_fia_species = df_fia_genus.copy()

In [ ]:
# Checking the distribution
df_fia_species['SPECIES'].value_counts(normalize=True) * 100

In [ ]:
# Split the 'tree_sp_scientific_name' column into two new columns using regex
df_field = pd.read_csv("data/03_tree.csv", low_memory=False)
df_field[['GENUS_NEW', 'SPECIES_NEW']] = df_field['tree_sp_scientific_name'].str.extract(r'(\S+)\s+(.*)')

# Display the updated dataframe
df_field.head()

In [ ]:
df_field['SPECIES_NEW'].value_counts()

In [ ]:
# List of species to keep
species_to_keep = ['decurrens', 'concolor', 'agrifolia', 'menziesii', 'jeffreyi',
                  'kelloggii', 'densiflorus', 'magnificar', 'ponderosa', 'douglasii',
                  'lambertiana', 'tremuloides', 'chrysolepis', 'sp.', 'nuttalii', 'lobata', 'giganteum', 'scouleriana', 'glabrum', 'macrophyllum']

# Filter the DataFrame
df_fia_species = df_fia_species[df_fia_species["SPECIES"].isin(species_to_keep)].copy()
df_fia_species.head()

In [ ]:
# Distribution of the species
df_fia_species['SPECIES'].value_counts()

In [ ]:
# Function to remove outliers based on IQR for each species
def remove_outliers_by_species(df, column, group_by):
    df_clean = df.copy()
    iteration = 0
    while True:
        iteration += 1
        Q1 = df_clean.groupby(group_by)[column].transform('quantile', 0.25)
        Q3 = df_clean.groupby(group_by)[column].transform('quantile', 0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Keep rows within bounds
        mask = (df_clean[column] >= lower_bound) & (df_clean[column] <= upper_bound)

        prev_shape = df_clean.shape[0]
        df_clean = df_clean[mask]
        new_shape = df_clean.shape[0]

        # If no further rows are removed, break
        if new_shape == prev_shape:
            print(f"No more outliers detected after {iteration} iterations.")
            break
        else:
            print(f"Iteration {iteration}: Removed {prev_shape - new_shape} rows.")

    return df_clean

columns = ['DIA', 'HT', 'BasalA', 'LAT', 'LON']

print(f"Shape before removing all outliers: {df_fia_species.shape}")

for column in columns:
    df_fia_species = remove_outliers_by_species(df_fia_species, column=column, group_by='SPECIES')

# Check resulting shape
print(f"Shape after removing all outliers: {df_fia_species.shape}")

In [ ]:
# make box plots to see outliers 
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.boxplot(x='SPECIES', y='DIA', data=df_fia_species)
plt.xticks(rotation=90)
plt.title('Boxplot of DIA by SPECIES')
plt.show()
plt.figure(figsize=(10, 6))

sns.boxplot(x='SPECIES', y='HT', data=df_fia_species)
plt.xticks(rotation=90)
plt.title('Boxplot of HT by SPECIES')
plt.show()
plt.figure(figsize=(10, 6))
sns.boxplot(x='SPECIES', y='BasalA', data=df_fia_species)
plt.xticks(rotation=90)
plt.title('Boxplot of BasalA by SPECIES')
plt.show()
plt.figure(figsize=(10, 6))
sns.boxplot(x='SPECIES', y='LAT', data=df_fia_species)
plt.xticks(rotation=90)
plt.title('Boxplot of LAT by SPECIES')
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='SPECIES', y='LON', data=df_fia_species)
plt.xticks(rotation=90)
plt.title('Boxplot of LON by SPECIES')
plt.show()

In [ ]:
# Selecting important features
include_vars = ['DIA', 'HT', 'LAT', 'LON', 'ECOSUBCD_261Ba', 'ECOSUBCD_263Am',
                'ECOSUBCD_M261Ej', 'ECOSUBCD_M261Em', 'PFT', 'SPECIES', 'GENUS']

df = df_fia_species[include_vars].dropna().copy()

# Create the encoder
ohe = OneHotEncoder(sparse_output=False)

# Fit and transform the PFT column
pft_encoded = ohe.fit_transform(df[["PFT"]])

# Convert to DataFrame with appropriate column names
pft_encoded_df = pd.DataFrame(
    pft_encoded,
    columns=[f"PFT_{cat}" for cat in ohe.categories_[0]],
    index=df.index
)

# Drop the original PFT column and join the encoded columns
df = df.drop(columns=["PFT"])
df = pd.concat([df, pft_encoded_df], axis=1)

#one-hot encode the GENUS column
genus_encoded = ohe.fit_transform(df[["GENUS"]])
genus_encoded_df = pd.DataFrame(
    genus_encoded,
    columns=[f"GENUS_{cat}" for cat in ohe.categories_[0]],
    index=df.index
)
# Drop the original GENUS column and join the encoded columns
df = df.drop(columns=["GENUS"])
df = pd.concat([df, genus_encoded_df], axis=1)


X = df.drop(columns=["SPECIES"])
y = df["SPECIES"]

# Encode target variable
le_species = LabelEncoder()
y_encoded = le_species.fit_transform(y)
num_classes = len(le_species.classes_)


# Stratified train-test split
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(
    X, y_encoded, test_size=0.30, stratify=y_encoded
)

from sklearn.compose import ColumnTransformer

# Identify numerical and categorical columns again
numerical_cols = ['DIA', 'HT', 'LAT', 'LON']
categorical_cols = [col for col in X.columns if col not in numerical_cols]

# Create column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_cols),
        # ("cat", OneHotEncoder(sparse_output=False), categorical_cols)
    ]
)

# Fit and transform training data
X_train_scaled = preprocessor.fit_transform(X_train)
X_test_scaled = preprocessor.transform(X_test)


# Concatenate scaled numerical and raw categorical columns
X_train_processed = np.hstack((X_train_scaled, X_train[categorical_cols].values))
X_test_processed = np.hstack((X_test_scaled, X_test[categorical_cols].values))

In [ ]:
# Compute sample weights
sample_weights = compute_sample_weight(class_weight='balanced', y=y_encoded)

# Check all shapes
print(f"X: {X.shape}, y: {len(y_encoded)}, sample_weights: {len(sample_weights)}")

In [ ]:
# Apply the column transformer to the dataset
X_new = np.hstack((preprocessor.fit_transform(X), X[categorical_cols].values))

In [ ]:
# Training the base learner - Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import time

# Best tuned model configuration
best_rf_model = RandomForestClassifier(
    class_weight={
        0: 3.6483766233766235,
        1: 0.6677561207511291,
        2: 0.25883355599576174,
        3: 0.4001210653753027,
        4: 9.909171075837742,
        5: 33.44345238095238,
        6: 1.182095518619819,
        7: 1.141739483844747,
        8: 1.182095518619819,
        9: 133.77380952380952,
        10: 17.448757763975156,
        11: 1.4462033462033461,
        12: 0.41869736940159474,
        13: 44.59126984126984
    },
    max_depth=100,
    max_features='log2',
    n_estimators=200,
    random_state=42
)

# Evaluation function
def train_and_evaluate(model, name):
    start_time = time.time()
    model.fit(X_train_processed, y_train_encoded)
    predictions = model.predict(X_test_processed)
    accuracy = accuracy_score(y_test_encoded, predictions)
    report = classification_report(y_test_encoded, predictions)
    end_time = time.time()
    training_time = end_time - start_time
    print(f"{name} - Accuracy: {accuracy:.4f}, Training Time: {training_time:.2f} seconds")
    print(report)
    print("-" * 50)

# Run final training + evaluation
train_and_evaluate(best_rf_model, "Best Random Forest (Final)")

#test the random forest model on the test set
predictions = best_rf_model.predict(X_test_processed)
accuracy = accuracy_score(y_test_encoded, predictions)
report = classification_report(y_test_encoded, predictions)
print(report)

In [ ]:
# Training the base learner - Extra Trees Classifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report, accuracy_score
import time

# Best tuned Extra Trees model config
best_et_model = ExtraTreesClassifier(
    n_estimators=600,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features=None,
    max_depth=None,
    criterion='gini',
    bootstrap=False,
    random_state=42
)

# Evaluation function
def train_and_evaluate(model, name):
    start_time = time.time()
    model.fit(X_train_processed, y_train_encoded)
    predictions = model.predict(X_test_processed)
    accuracy = accuracy_score(y_test_encoded, predictions)
    report = classification_report(y_test_encoded, predictions)
    end_time = time.time()
    training_time = end_time - start_time
    print(f"{name} - Accuracy: {accuracy:.4f}, Training Time: {training_time:.2f} seconds")
    print(report)
    print("-" * 50)

# Run final training + evaluation
train_and_evaluate(best_et_model, "Best Extra Trees (Final)")

predictions = best_et_model.predict(X_test_processed)
accuracy = accuracy_score(y_test_encoded, predictions)
report = classification_report(y_test_encoded, predictions)
print(report)

In [ ]:
# Training the base learner - Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
import time

# Best tuned Decision Tree model config
best_dt_model = DecisionTreeClassifier(
    criterion='entropy',
    max_depth=50,
    min_samples_leaf=4,
    min_samples_split=15,
    random_state=42
)

# Evaluation function (assumed already defined)
def train_and_evaluate(model, name):
    start_time = time.time()
    model.fit(X_train_processed, y_train_encoded)
    predictions = model.predict(X_test_processed)
    accuracy = accuracy_score(y_test_encoded, predictions)
    report = classification_report(y_test_encoded, predictions)
    end_time = time.time()
    training_time = end_time - start_time
    print(f"{name} - Accuracy: {accuracy:.4f}, Training Time: {training_time:.2f} seconds")
    print(report)
    print("-" * 50)

# Train + evaluate the final Decision Tree model
train_and_evaluate(best_dt_model, "Best Decision Tree (Final)")

predictions = best_dt_model.predict(X_test_processed)
accuracy = accuracy_score(y_test_encoded, predictions)
report = classification_report(y_test_encoded, predictions)
print(report)

In [ ]:
# Training the meta-learner - Stacking with XGBoost
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import classification_report, accuracy_score
import time

# Reuse base models
base_models = [
    ('rf', best_rf_model),
    ('et', best_et_model),
    ('dt', best_dt_model)
]

# Meta-learner with best-found XGB config
best_xgb_meta = XGBClassifier(
    n_estimators=50,
    learning_rate=0.01,
    max_depth=7,
    min_child_weight=2,
    subsample=1.0,               
    colsample_bytree=0.9,
    gamma=0,
    reg_alpha=0,
    reg_lambda=1,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

# Final stacking model
best_stack_xgb_model = StackingClassifier(
    estimators=base_models,
    final_estimator=best_xgb_meta,
    cv=5,
    n_jobs=-1
)

# Evaluation function
def train_and_evaluate(model, name):
    start_time = time.time()
    model.fit(X_train_processed, y_train_encoded)
    predictions = model.predict(X_test_processed)
    accuracy = accuracy_score(y_test_encoded, predictions)
    report = classification_report(y_test_encoded, predictions)
    end_time = time.time()
    training_time = end_time - start_time
    print(f"{name} - Accuracy: {accuracy:.4f}, Training Time: {training_time:.2f} seconds")
    print(report)
    print("-" * 50)

# Train + evaluate final stacking model
train_and_evaluate(best_stack_xgb_model, "Final Stacking with XGBoost")


In [ ]:
# Fitting the data to final stacked model
best_stack_xgb_model.fit(X_new,y_encoded, sample_weight=sample_weights)

In [ ]:
# Evaluation
predictions = best_stack_xgb_model.predict(X_test_processed)
accuracy = accuracy_score(y_test_encoded, predictions)
predictions = le_species.inverse_transform(predictions)
y_test_labels = le_species.inverse_transform(y_test_encoded)  # Transform y_test_encoded to original labels
report = classification_report(y_test_labels, predictions)  # Use transformed labels
print(report)

In [ ]:
df_field['SPECIES_NEW'].value_counts(normalize=True) * 100

### Working with TLS data

In [ ]:
# One-hot encode the PFT column
genus_encoded = pd.get_dummies(TLS_species['predicted_genus'], prefix='GENUS')

# Concatenate the one-hot encoded columns with the original DataFrame
TLS_species = pd.concat([TLS_species, genus_encoded], axis=1)

# Drop the original PFT column
TLS_species.drop(columns=['predicted_genus'], inplace=True)

# Display the updated DataFrame
TLS_species.head()

In [ ]:
# Adding these two columns as they are not predicted by the model and settinhg them to false to match the training dimensions
# This still means that the genus for current model is not Arbutus, Lithocarpus, Populus
TLS_species[['GENUS_Arbutus', 'GENUS_Sequoiadendron', 'GENUS_Populus']] = False, False, False

In [ ]:
# Selecting the features for model prediction
TLS_species_features = TLS_species[['DIA', 'HT', 'LAT', 'LON', 'ECOSUBCD_261Ba', 'ECOSUBCD_263Am','ECOSUBCD_M261Ej', 'ECOSUBCD_M261Em', 'PFT_Deciduous broadleaf',
       'PFT_Evergreen broadleaf', 'PFT_Evergreen conifer', 'GENUS_Abies',
       'GENUS_Acer', 'GENUS_Calocedrus', 'GENUS_Pinus', 'GENUS_Pseudotsuga',
       'GENUS_Quercus', 'GENUS_Sequoiadendron', 'GENUS_Arbutus',
       'GENUS_Populus']]
len(TLS_species_features)

In [ ]:
categorical_cols = [col for col in TLS_species_features.columns if col not in numerical_cols]
TLS_species_features_array = np.hstack((preprocessor.fit_transform(TLS_species), TLS_species[categorical_cols].values))
#predictions
pred =  best_stack_xgb_model.predict(TLS_species_features_array)
predicted_species = le_species.inverse_transform(pred)
TLS_species['predicted_species'] = predicted_species.copy()
TLS_species['predicted_species'].value_counts()


In [ ]:
print("Training data distribution:")
print(df_fia_species['SPECIES'].value_counts(normalize=True) * 100)

In [ ]:
print("-" * 30)

# find percentage of each predicted genus
predicted_species_counts = TLS_species['predicted_species'].value_counts(normalize=True) * 100
# Display the percentage of each predicted genus
print("Percentage of each predicted species:")
print(predicted_species_counts)
print("-" * 30)

print("Field data distribution")
print(df_field["SPECIES_NEW"].value_counts(normalize=True) * 100)


Here, we have decurrens predicted as 25% (Predicted) to 22% (Field data) and concolor coming in close at 19% (Predicted) to 20% (Field)